## Definindo o Problema

In [22]:
#!pip install lightgbm;
#!pip install pymoo;
#!pip install -U pyrecorder;

In [23]:
ITERATIONS = 32
POPULATION = 32

In [24]:
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
#from yellowbrick.classifier import ConfusionMatrix
import numpy as np
from tqdm.notebook import tqdm as tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, KFold
#from google.colab import drive
import pickle
from pymoo.algorithms.soo.nonconvex.pso import PSO, PSOAnimation
#from pymoo.factory import Rastrigin
from pymoo.optimize import minimize
import matplotlib.pyplot as plt
from pymoo.factory import get_termination
#from pymoo.util.display import Display
from pymoo.core.callback import Callback

In [25]:
from catboost import CatBoostClassifier

In [26]:
#drive.mount('/content/gdrive')

In [27]:
#with open('/content/gdrive/MyDrive/datasets/credit.pkl', 'rb') as f:
#    x_fruit_train, x_fruit_test, y_fruit_train, y_fruit_test = pickle.load(f)
    
#with open('./fruit.pkl', 'rb') as f:
#    x_fruit, y_fruit = pickle.load(f)
    
with open('./fruit.pkl', 'rb') as f:
    x_fruit, y_fruit = pickle.load(f)

Definindo as restrições 

In [28]:
DIMENSIONS = 16

In [29]:
iterations_min = 20  #n_estimators n_estimators #INT
learning_rate_min = 0.001  #FLOAT
l2_leaf_reg_min = 0.01  #FLOAT reg_lambda
bagging_temperature_min = 0.001  # float
subsample_min = 1.0   #float 
mvs_reg_min = 0.001  #float 0 - inf
random_strength_min = 0.001  #float 1.0 
use_best_model_min = False  #bool true false
best_model_min_trees_min = 1  #int 
depth_min = 1  #max_depth 8 int
min_data_in_leaf_min = 1  #min_child_samples int
max_leaves_min = 10  #num_leaves int 
one_hot_max_size_min = 1  #int 
rsm_min = 0.001  #colsample_bylevel float (0;1]
fold_permutation_block_min = 1  #int 
leaf_estimation_iterations_min = 1  #int
fold_len_multiplier_min = 2.0  #float
approx_on_full_history_min = False  #bool 
boost_from_average_min = False  #bool 
diffusion_temperature_min = 1000  #float 10000
posterior_sampling_min = False  #bool
allow_const_label_min = False  #bool
penalties_coefficient_min = 1.0  #float
model_shrink_rate_min = 0.001  #float

iterations_max = 100  #n_estimators n_estimators #INT
learning_rate_max = 0.5  #FLOAT
l2_leaf_reg_max = 1  #FLOAT reg_lambda
bagging_temperature_max = 10000.0  # float
subsample_max = 2.0  #float 
mvs_reg_max = 1000.0  #float 0 - inf
random_strength_max = 10.0  #float 1.0 
use_best_model_max = True  #bool true false
best_model_min_trees_max = 10  #int 
depth_max = 1  #max_depth 8 int 
min_data_in_leaf_max = 100  #min_child_samples int
max_leaves_max = 10  #num_leaves int 
one_hot_max_size_max = 20  #int 
rsm_max = 1.0   #colsample_bylevel float (0;1]
fold_permutation_block_max = 100  #int 
leaf_estimation_iterations_max = 100  #int
fold_len_multiplier_max = 10.0  #float
approx_on_full_history_max = True  #bool 
boost_from_average_max = True  #bool 
diffusion_temperature_max = 50000  #float 10000
posterior_sampling_max = True  #bool
allow_const_label_max = True  #bool
penalties_coefficient_max = 10.0  #float
model_shrink_rate_max = 1.0  #float

grow_policy = 'Lossguide'

Definindo o problema

In [30]:
import numpy as np
from pymoo.core.problem import ElementwiseProblem

class OptimizeWithAccuracy(ElementwiseProblem):

    def __init__(self):
        super().__init__(n_var= DIMENSIONS,
                         n_obj=1,
                         n_constr=16,
                         xl=np.array([iterations_min,
                          learning_rate_min,
                          l2_leaf_reg_min,
                          bagging_temperature_min,
                          #subsample_min,
                          #mvs_reg_min,
                          random_strength_min,
                          #use_best_model_min,
                          best_model_min_trees_min,
                          depth_min,
                          min_data_in_leaf_min,
                          max_leaves_min,
                          one_hot_max_size_min,
                          rsm_min,
                          fold_permutation_block_min,
                          leaf_estimation_iterations_min,
                          fold_len_multiplier_min,
                          #approx_on_full_history_min,
                          #boost_from_average_min,
                          #diffusion_temperature_min,
                          #posterior_sampling_min,
                          #allow_const_label_min,
                          penalties_coefficient_min,
                          model_shrink_rate_min]),
                         xu=np.array([iterations_max,
                            learning_rate_max,
                            l2_leaf_reg_max,
                            bagging_temperature_max,
                            #subsample_max,
                            #mvs_reg_max,
                            random_strength_max,
                            #use_best_model_max,
                            best_model_min_trees_max,
                            depth_max,
                            min_data_in_leaf_max,
                            max_leaves_max,
                            one_hot_max_size_max,
                            rsm_max,
                            fold_permutation_block_max,
                            leaf_estimation_iterations_max,
                            fold_len_multiplier_max,
                            #approx_on_full_history_max,
                            #boost_from_average_max,
                            #diffusion_temperature_max,
                            #posterior_sampling_max,
                            #allow_const_label_max,
                            penalties_coefficient_max,
                            model_shrink_rate_max])
                        )

    def _evaluate(self, x, out, *args, **kwargs):
        #num_leaves, min_child_samples, n_estimators, learning_rate, subsample_for_bin, min_split_gain, min_child_weight, reg_alpha, reg_lambda
        
        model_catboost = CatBoostClassifier(#num_leaves        = int(np.round(x[0])), 
                                       #extra_trees = bool(np.round(x[16])), #boolean
                                        iterations = int(np.round(x[0])),
                                        
                                        learning_rate = x[1],
                                        l2_leaf_reg = x[2],
                                        bagging_temperature = x[3],
                                        #subsample = x[4],
                                        #mvs_reg = x[5],
                                        random_strength = x[4],
                                        #use_best_model = bool(np.round(x[7])),
                                        best_model_min_trees = int(np.round(x[5])), 
                                        depth = int(np.round(x[6])), 
                                        min_data_in_leaf = int(np.round(x[7])), 
                                        max_leaves = int(np.round(x[8])), 
                                        one_hot_max_size = int(np.round(x[9])), 
                                        rsm = x[10],
                                        fold_permutation_block = int(np.round(x[11])), 
                                        leaf_estimation_iterations = int(np.round(x[12])), 
                                        fold_len_multiplier = x[13],
                                        approx_on_full_history = False, #bool(np.round(x[17])),
                                        #boost_from_average = bool(np.round(x[18])),
                                        #diffusion_temperature = x[19],
                                        #posterior_sampling = bool(np.round(x[20])),
                                        #allow_const_label = bool(np.round(x[21])),
                                        penalties_coefficient = x[14],
                                        model_shrink_rate = x[15],
                                        loss_function='MultiClass',
                                        grow_policy = 'Lossguide',
                                        thread_count = -1
                                       )
        
        kfold = KFold(n_splits = 3, shuffle = True)
        
        scores = cross_val_score(model_catboost, x_fruit, y_fruit, cv = kfold, n_jobs=-1)  
        result = scores.mean()
        #if not result:
        #  result = 0
        
        out['F'] = -1 * result

problemAccuracy = OptimizeWithAccuracy()

In [31]:
class OptimizeWithF1(ElementwiseProblem):
        def __init__(self):
                super().__init__(n_var= DIMENSIONS,
                         n_obj=1,
                         n_constr=25,
                         xl=np.array([iterations_min,
                          learning_rate_min,
                          l2_leaf_reg_min,
                          bagging_temperature_min,
                          subsample_min,
                          mvs_reg_min,
                          random_strength_min,
                          use_best_model_min,
                          best_model_min_trees_min,
                          depth_min,
                          min_data_in_leaf_min,
                          max_leaves_min,
                          one_hot_max_size_min,
                          rsm_min,
                          fold_permutation_block_min,
                          leaf_estimation_iterations_min,
                          fold_len_multiplier_min,
                          approx_on_full_history_min,
                          boost_from_average_min,
                          diffusion_temperature_min,
                          posterior_sampling_min,
                          allow_const_label_min,
                          penalties_coefficient_min,
                          model_shrink_rate_min]),
                         xu=np.array([iterations_max,
                            learning_rate_max,
                            l2_leaf_reg_max,
                            bagging_temperature_max,
                            subsample_max,
                            mvs_reg_max,
                            random_strength_max,
                            use_best_model_max,
                            best_model_min_trees_max,
                            depth_max,
                            min_data_in_leaf_max,
                            max_leaves_max,
                            one_hot_max_size_max,
                            rsm_max,
                            fold_permutation_block_max,
                            leaf_estimation_iterations_max,
                            fold_len_multiplier_max,
                            approx_on_full_history_max,
                            boost_from_average_max,
                            diffusion_temperature_max,
                            posterior_sampling_max,
                            allow_const_label_max,
                            penalties_coefficient_max,
                            model_shrink_rate_max])
                        )

        def _evaluate(self, x, out, *args, **kwargs):
                #num_leaves, min_child_samples, n_estimators, learning_rate, subsample_for_bin, min_split_gain, min_child_weight, reg_alpha, reg_lambda
                
                model_catboost = CatBoostClassifier(#num_leaves        = int(np.round(x[0])), 
                                        #extra_trees = bool(np.round(x[16])), #boolean
                                                #extra_trees = bool(np.round(x[16])), #boolean
                                        iterations = int(np.round(x[0])),
                                        
                                        learning_rate = x[1],
                                        l2_leaf_reg = x[2],
                                        bagging_temperature = x[3],
                                        #subsample = x[4],
                                        #mvs_reg = x[5],
                                        random_strength = x[6],
                                        #use_best_model = bool(np.round(x[7])),
                                        best_model_min_trees = int(np.round(x[8])), 
                                        depth = int(np.round(x[9])), 
                                        min_data_in_leaf = int(np.round(x[10])), 
                                        max_leaves = int(np.round(x[11])), 
                                        one_hot_max_size = int(np.round(x[12])), 
                                        rsm = x[13],
                                        fold_permutation_block = int(np.round(x[14])), 
                                        leaf_estimation_iterations = int(np.round(x[15])), 
                                        fold_len_multiplier = x[16],
                                        approx_on_full_history = False, #bool(np.round(x[17])),
                                        #boost_from_average = bool(np.round(x[18])),
                                        #diffusion_temperature = x[19],
                                        posterior_sampling = bool(np.round(x[20])),
                                        allow_const_label = bool(np.round(x[21])),
                                        penalties_coefficient = x[22],
                                        model_shrink_rate = x[23],
                                        loss_function='MultiClass',
                                        grow_policy = 'Lossguide',
                                        thread_count = -1
                                        )
                
                kfold = KFold(n_splits = 3, shuffle = True)

                scores = cross_val_score(model_catboost, x_fruit, y_fruit, cv = kfold, scoring='f1_weighted', n_jobs=-1)  
                result = scores.mean()
                out['F'] = -1 * result

problemF1 = OptimizeWithF1()

In [32]:
class OptimizeWithAUC(ElementwiseProblem):
    def __init__(self):
        super().__init__(n_var= DIMENSIONS,
                         n_obj=1,
                         n_constr=25,
                         xl=np.array([iterations_min,
                          learning_rate_min,
                          l2_leaf_reg_min,
                          bagging_temperature_min,
                          subsample_min,
                          mvs_reg_min,
                          random_strength_min,
                          use_best_model_min,
                          best_model_min_trees_min,
                          depth_min,
                          min_data_in_leaf_min,
                          max_leaves_min,
                          one_hot_max_size_min,
                          rsm_min,
                          fold_permutation_block_min,
                          leaf_estimation_iterations_min,
                          fold_len_multiplier_min,
                          approx_on_full_history_min,
                          boost_from_average_min,
                          diffusion_temperature_min,
                          posterior_sampling_min,
                          allow_const_label_min,
                          penalties_coefficient_min,
                          model_shrink_rate_min]),
                         xu=np.array([iterations_max,
                            learning_rate_max,
                            l2_leaf_reg_max,
                            bagging_temperature_max,
                            subsample_max,
                            mvs_reg_max,
                            random_strength_max,
                            use_best_model_max,
                            best_model_min_trees_max,
                            depth_max,
                            min_data_in_leaf_max,
                            max_leaves_max,
                            one_hot_max_size_max,
                            rsm_max,
                            fold_permutation_block_max,
                            leaf_estimation_iterations_max,
                            fold_len_multiplier_max,
                            approx_on_full_history_max,
                            boost_from_average_max,
                            diffusion_temperature_max,
                            posterior_sampling_max,
                            allow_const_label_max,
                            penalties_coefficient_max,
                            model_shrink_rate_max])
                        )

    def _evaluate(self, x, out, *args, **kwargs):
        #num_leaves, min_child_samples, n_estimators, learning_rate, subsample_for_bin, min_split_gain, min_child_weight, reg_alpha, reg_lambda
        
        model_catboost = CatBoostClassifier(#num_leaves        = int(np.round(x[0])), 
                                       #extra_trees = bool(np.round(x[16])), #boolean
                                        #extra_trees = bool(np.round(x[16])), #boolean
                                        iterations = int(np.round(x[0])),
                                        
                                        learning_rate = x[1],
                                        l2_leaf_reg = x[2],
                                        bagging_temperature = x[3],
                                        #subsample = x[4],
                                        #mvs_reg = x[5],
                                        random_strength = x[6],
                                        #use_best_model = bool(np.round(x[7])),
                                        best_model_min_trees = int(np.round(x[8])), 
                                        depth = int(np.round(x[9])), 
                                        min_data_in_leaf = int(np.round(x[10])), 
                                        max_leaves = int(np.round(x[11])), 
                                        one_hot_max_size = int(np.round(x[12])), 
                                        rsm = x[13],
                                        fold_permutation_block = int(np.round(x[14])), 
                                        leaf_estimation_iterations = int(np.round(x[15])), 
                                        fold_len_multiplier = x[16],
                                        approx_on_full_history = False, #bool(np.round(x[17])),
                                        #boost_from_average = bool(np.round(x[18])),
                                        #diffusion_temperature = x[19],
                                        posterior_sampling = bool(np.round(x[20])),
                                        allow_const_label = bool(np.round(x[21])),
                                        penalties_coefficient = x[22],
                                        model_shrink_rate = x[23],
                                        loss_function='MultiClass',
                                        grow_policy = 'Lossguide',
                                        thread_count = -1
                                       )
        
        kfold = KFold(n_splits = 3, shuffle = True)
   
        scores = cross_val_score(model_catboost, x_fruit, y_fruit, cv = kfold, scoring='roc_auc_ovr_weighted', n_jobs=-1)  
        
        result = scores.mean()

        out['F'] = -1 * result

problemAUC = OptimizeWithAUC()

In [33]:
from pymoo.util.display.column import Column
from pymoo.util.display.output import Output

In [34]:
class MyOutput(Output):

    def __init__(self):
        super().__init__()
        global pbar 
        pbar = tqdm(total=ITERATIONS)
        self.score = Column("score", width=13)
        self.Parameters = Column("Parameters", width=35)
        self.columns += [self.score, self.Parameters]

    def update(self, algorithm):
        super().update(algorithm)
        self.score.set(-np.min(algorithm.pop.get("F")))
        #self.Parameters.set(algorithm.pop.get("X")[0])
        pbar.update(1)
        if pbar.n == ITERATIONS: pbar.close()

## Particle Swarm Optimization (PSO)

### Acurácia

In [35]:
iterations_min = 20  #n_estimators n_estimators #INT
learning_rate_min = 0.001  #FLOAT
l2_leaf_reg_min = 0.01  #FLOAT reg_lambda
bagging_temperature_min = 0.001  # float
subsample_min = 1.0   #float 
mvs_reg_min = 0.001  #float 0 - inf
random_strength_min = 0.001  #float 1.0 
use_best_model_min = False  #bool true false
best_model_min_trees_min = 1  #int 
depth_min = 1  #max_depth 8 int
min_data_in_leaf_min = 1  #min_child_samples int
max_leaves_min = 1  #num_leaves int 
one_hot_max_size_min = 1  #int 
rsm_min = 0.001  #colsample_bylevel float (0;1]
fold_permutation_block_min = 1  #int 
leaf_estimation_iterations_min = 1  #int
fold_len_multiplier_min = 2.0  #float
approx_on_full_history_min = False  #bool 
boost_from_average_min = False  #bool 
diffusion_temperature_min = 1000  #float 10000
posterior_sampling_min = False  #bool
allow_const_label_min = False  #bool
penalties_coefficient_min = 1.0  #float
model_shrink_rate_min = 0.001  #float

iterations_max = 100  #n_estimators n_estimators #INT
learning_rate_max = 0.5  #FLOAT
l2_leaf_reg_max = 1  #FLOAT reg_lambda
bagging_temperature_max = 10000.0  # float
subsample_max = 2.0  #float 
mvs_reg_max = 1000.0  #float 0 - inf
random_strength_max = 10.0  #float 1.0 
use_best_model_max = True  #bool true false
best_model_min_trees_max = 10  #int 
depth_max = 8  #max_depth 8 int 
min_data_in_leaf_max = 100  #min_child_samples int
max_leaves_max = 10  #num_leaves int 
one_hot_max_size_max = 20  #int 
rsm_max = 1.0   #colsample_bylevel float (0;1]
fold_permutation_block_max = 100  #int 
leaf_estimation_iterations_max = 100  #int
fold_len_multiplier_max = 10.0  #float
approx_on_full_history_max = True  #bool 
boost_from_average_max = True  #bool 
diffusion_temperature_max = 50000  #float 10000
posterior_sampling_max = True  #bool
allow_const_label_max = True  #bool
penalties_coefficient_max = 10.0  #float
model_shrink_rate_max = 1.0  #float

grow_policy = 'Lossguide'

In [36]:
ITERATIONS = 32
POPULATION = 32

In [37]:
import pyswarms as ps
from pyswarms.utils.functions import single_obj as fx
import numpy as np

In [38]:
xl=np.array([iterations_min,
            learning_rate_min,
            l2_leaf_reg_min,
            bagging_temperature_min,
            random_strength_min,
            best_model_min_trees_min,
            depth_min,
            min_data_in_leaf_min,
            max_leaves_min,
            one_hot_max_size_min,
            rsm_min,
            fold_permutation_block_min,
            leaf_estimation_iterations_min,
            fold_len_multiplier_min,
            penalties_coefficient_min,
            model_shrink_rate_min])
xu=np.array([iterations_max,
            learning_rate_max,
            l2_leaf_reg_max,
            bagging_temperature_max,
            random_strength_max,
            best_model_min_trees_max,
            depth_max,
            min_data_in_leaf_max,
            max_leaves_max,
            one_hot_max_size_max,
            rsm_max,
            fold_permutation_block_max,
            leaf_estimation_iterations_max,
            fold_len_multiplier_max,
            penalties_coefficient_max,
            model_shrink_rate_max])

In [39]:
def PSO_Optimize_Accuracy(values):
    x = values[0] 
    model_catboost = CatBoostClassifier(
                                    iterations = int(np.round(x[0])),
                                    learning_rate = x[1],
                                    l2_leaf_reg = x[2],
                                    bagging_temperature = x[3],
                                    random_strength = x[4],
                                    best_model_min_trees = int(np.round(x[5])), 
                                    depth = int(np.round(x[6])), 
                                    min_data_in_leaf = int(np.round(x[7])), 
                                    max_leaves = int(np.round(x[8])), 
                                    one_hot_max_size = int(np.round(x[9])), 
                                    rsm = x[10],
                                    fold_permutation_block = int(np.round(x[11])), 
                                    leaf_estimation_iterations = int(np.round(x[12])), 
                                    fold_len_multiplier = x[13],
                                    approx_on_full_history = False,
                                    penalties_coefficient = x[14],
                                    model_shrink_rate = x[15],
                                  
                                    grow_policy = 'Lossguide',
                                    thread_count = -1
                                    )
    
    kfold = KFold(n_splits = 10, shuffle = True)
    
    scores = cross_val_score(model_catboost, x_fruit, y_fruit, cv = kfold, n_jobs=-1)  
    result = scores.mean()     

    return -result

In [40]:
def PSO_Optimize_F1(values):
    x = values[0] 
    model_catboost = CatBoostClassifier(
                                    iterations = int(np.round(x[0])),
                                    learning_rate = x[1],
                                    l2_leaf_reg = x[2],
                                    bagging_temperature = x[3],
                                    random_strength = x[4],
                                    best_model_min_trees = int(np.round(x[5])), 
                                    depth = int(np.round(x[6])), 
                                    min_data_in_leaf = int(np.round(x[7])), 
                                    max_leaves = int(np.round(x[8])), 
                                    one_hot_max_size = int(np.round(x[9])), 
                                    rsm = x[10],
                                    fold_permutation_block = int(np.round(x[11])), 
                                    leaf_estimation_iterations = int(np.round(x[12])), 
                                    fold_len_multiplier = x[13],
                                    approx_on_full_history = False,
                                    penalties_coefficient = x[14],
                                    model_shrink_rate = x[15],
               
                                    grow_policy = 'Lossguide',
                                    thread_count = -1
                                    )
    
    kfold = KFold(n_splits = 10, shuffle = True)
    
    scores = cross_val_score(model_catboost,  x_fruit, y_fruit, cv = kfold, n_jobs=-1, scoring='f1_weighted')  
    result = scores.mean()     

    return -result

In [41]:
def PSO_Optimize_AUC(values):
    x = values[0] 
    model_catboost = CatBoostClassifier(
                                    iterations = int(np.round(x[0])),
                                    learning_rate = x[1],
                                    l2_leaf_reg = x[2],
                                    bagging_temperature = x[3],
                                    random_strength = x[4],
                                    best_model_min_trees = int(np.round(x[5])), 
                                    depth = int(np.round(x[6])), 
                                    min_data_in_leaf = int(np.round(x[7])), 
                                    max_leaves = int(np.round(x[8])), 
                                    one_hot_max_size = int(np.round(x[9])), 
                                    rsm = x[10],
                                    fold_permutation_block = int(np.round(x[11])), 
                                    leaf_estimation_iterations = int(np.round(x[12])), 
                                    fold_len_multiplier = x[13],
                                    approx_on_full_history = False,
                                    penalties_coefficient = x[14],
                                    model_shrink_rate = x[15],
                           
                                    grow_policy = 'Lossguide',
                                    thread_count = -1
                                    )
    
    kfold = KFold(n_splits = 10, shuffle = True)
    
    scores = cross_val_score(model_catboost, x_fruit, y_fruit, cv = kfold, n_jobs=-1, scoring='roc_auc_ovr_weighted')  
    result = scores.mean()     

    return -result

In [42]:
def run_accuracy_pso():
    # Call an instance of PSO
    swarm_size = 32
    iters = 32
    dim = 16
    options = {'c1': 1.5, 'c2':1.5, 'w':0.5}
    constraints = (xl,xu)

    optimizer = ps.single.GlobalBestPSO(n_particles=swarm_size,
                                        dimensions=dim,
                                        options=options,
                                        bounds=constraints)
    cost, joint_vars = optimizer.optimize(objective_func = PSO_Optimize_Accuracy, iters=iters)
    return -cost

In [43]:
run_accuracy_pso()

2023-05-30 21:05:45,102 - pyswarms.single.global_best - INFO - Optimize for 32 iters with {'c1': 1.5, 'c2': 1.5, 'w': 0.5}
pyswarms.single.global_best: 100%|██████████|32/32, best_cost=-.219
2023-05-30 21:06:10,950 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -0.21933832709113604, best pos: [8.29951455e+01 3.19234849e-01 1.99754334e-01 7.55022262e+03
 3.47527242e+00 2.64115377e+00 3.16050323e+00 6.22588593e+01
 7.35995791e+00 1.27938748e+01 1.99666352e-01 6.27239006e+01
 9.29199370e+00 7.77401794e+00 8.65720708e+00 4.48917312e-01]


0.21933832709113604

In [44]:
def run_auc_pso():
    # Call an instance of PSO
    swarm_size = 32
    iters = 32
    dim = 16
    options = {'c1': 1.5, 'c2':1.5, 'w':0.5}
    constraints = (xl,xu)

    optimizer = ps.single.GlobalBestPSO(n_particles=swarm_size,
                                        dimensions=dim,
                                        options=options,
                                        bounds=constraints)
    cost, joint_vars = optimizer.optimize(objective_func = PSO_Optimize_AUC, iters=iters)
    return -cost

In [45]:
run_auc_pso()

2023-05-30 21:06:11,283 - pyswarms.single.global_best - INFO - Optimize for 32 iters with {'c1': 1.5, 'c2': 1.5, 'w': 0.5}
pyswarms.single.global_best: 100%|██████████|32/32, best_cost=-.5
2023-05-30 21:06:32,369 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -0.5, best pos: [9.34064466e+01 4.75298282e-01 4.87568503e-01 4.67254467e+03
 5.16959742e+00 3.77826115e+00 7.92488365e+00 1.25455232e+01
 8.93476589e+00 1.16078782e+01 4.87230264e-01 3.18550245e+01
 2.77204506e+00 8.43894630e+00 9.02109922e+00 8.43101286e-02]


0.5

In [46]:
def run_f1_pso():
    # Call an instance of PSO
    swarm_size = 32
    iters = 32
    dim = 16
    options = {'c1': 1.5, 'c2':1.5, 'w':0.5}
    constraints = (xl,xu)

    optimizer = ps.single.GlobalBestPSO(n_particles=swarm_size,
                                        dimensions=dim,
                                        options=options,
                                        bounds=constraints)
    cost, joint_vars = optimizer.optimize(objective_func = PSO_Optimize_F1, iters=iters)
    return -cost

In [47]:
run_f1_pso()

2023-05-30 21:06:32,655 - pyswarms.single.global_best - INFO - Optimize for 32 iters with {'c1': 1.5, 'c2': 1.5, 'w': 0.5}
pyswarms.single.global_best: 100%|██████████|32/32, best_cost=-.0819
2023-05-30 21:07:11,705 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -0.08193206255385381, best pos: [9.21500289e+01 9.71404560e-02 9.60036004e-02 5.51934893e+03
 3.15566460e+00 3.58977779e+00 5.20800708e+00 6.65082483e+01
 4.80659913e+00 1.78537232e+01 6.90713093e-01 3.77211864e+01
 2.57539325e+01 9.69762815e+00 5.42168341e+00 6.02323383e-01]


0.08193206255385381

## Algoritmo Genético (GA)

### Acurácia

In [48]:
ITERATIONS = 32
POPULATION = 32

In [49]:
from pymoo.algorithms.soo.nonconvex.ga import GA

In [50]:
def run_accuracy_ga(ITERATIONS = 32, POPULATION = 32):

    algorithm = GA(pop_size=POPULATION)

    term = get_termination("n_gen", ITERATIONS)

    res = minimize(problemAccuracy,
                algorithm,
                save_history=False,
                verbose=True,
                output=MyOutput(),
                termination = term)


    index_best_individual = np.where(res.pop.get('F') == np.min(res.pop.get('F')))[0][0]
    score_best_individual = res.pop.get('F')[index_best_individual]
    parameters_best_individual = res.pop.get('X')[index_best_individual]

    #print(f'Best Accuracy Score {-score_best_individual}')
    #print(f'Model parameters: \n {parameters_best_individual}')
    
    return score_best_individual, parameters_best_individual, res

In [51]:
score_best_individual, parameters_best_individual, res = run_accuracy_ga(ITERATIONS = 15, POPULATION = 5)

C:\Users\Felps\AppData\Local\Temp\ipykernel_10512\2209151331.py:5: DeprecationWarning: Call to deprecated function (or staticmethod) get_termination. (Please use `from pymoo.termination import get_termination`)
  term = get_termination("n_gen", ITERATIONS)


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
#tracking = [-np.min(individual.pop.get('F')) for individual in res.history ]
#tracking_GA_Accuracy = tracking
#plt.plot(tracking)

In [ ]:
#trlist = np.array([])
#for i in range(len(res.history)):
#  trlist = np.append(trlist, -res.history[i].pop.get('F').reshape(-1)) 

In [ ]:
#scorelist = []
#
#for i in range(len(trlist)):
#  if i == 0:
#    scorelist.append(trlist[i])
#  elif trlist[i] > scorelist[i-1]:
#    scorelist.append(trlist[i])
#  else:
#    scorelist.append(scorelist[i-1])

In [ ]:
#Accuracy_GA = scorelist

In [ ]:
#plt.plot(scorelist);

### F1 Score 

In [ ]:
from pymoo.algorithms.soo.nonconvex.ga import GA
def run_f1_ga(ITERATIONS = 32, POPULATION = 32):
    algorithm = GA(pop_size=POPULATION)

    term = get_termination("n_gen", ITERATIONS)

    res = minimize(problemF1,
                algorithm,
                save_history=False,
                verbose=False,
                output=MyOutput(),
                termination = term)


    index_best_individual = np.where(res.pop.get('F') == np.min(res.pop.get('F')))[0][0]
    score_best_individual = res.pop.get('F')[index_best_individual]
    parameters_best_individual = res.pop.get('X')[index_best_individual]

    #print(f'Best F1 Score {-score_best_individual}')
    #print(f'Model parameters: \n {parameters_best_individual}')
    
    return score_best_individual, parameters_best_individual, res

In [ ]:
score_best_individual, parameters_best_individual, res = run_f1_ga(ITERATIONS = 10, POPULATION = 1)

C:\Users\Felps\AppData\Local\Temp\ipykernel_12056\1994123235.py:5: DeprecationWarning: Call to deprecated function (or staticmethod) get_termination. (Please use `from pymoo.termination import get_termination`)
  term = get_termination("n_gen", ITERATIONS)


  0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
#trlist = np.array([])
#for i in range(len(res.history)):
#  trlist = np.append(trlist, -res.history[i].pop.get('F').reshape(-1)) 
#  
#scorelist = []
#
#for i in range(len(trlist)):
##for i in range(1):
#  if i == 0:
#    scorelist.append(trlist[i])
#  elif trlist[i] > scorelist[i-1]:
#    scorelist.append(trlist[i])
#  else:
#    scorelist.append(scorelist[i-1])
#    
#F1_GA = scorelist
#
#plt.plot(scorelist);

In [ ]:
#tracking = [-np.min(individual.pop.get('F')) for individual in res.history ]
#tracking_GA_F1 = tracking
#plt.plot(tracking)

### AUC

In [ ]:
from pymoo.algorithms.soo.nonconvex.ga import GA
def run_auc_ga(ITERATIONS = 32, POPULATION = 32):
    algorithm = GA(pop_size=POPULATION)

    term = get_termination("n_gen", ITERATIONS)

    res = minimize(problemAUC,
                algorithm,
                save_history=False,
                verbose=False,
                output=MyOutput(),
                termination = term)

    
    index_best_individual = np.where(res.pop.get('F') == np.min(res.pop.get('F')))[0][0]
    score_best_individual = res.pop.get('F')[index_best_individual]
    parameters_best_individual = res.pop.get('X')[index_best_individual]

    #print(f'Best AUC Score {-score_best_individual}')
    #print(f'Model parameters: \n {parameters_best_individual}')
    
    return score_best_individual, parameters_best_individual, res

In [ ]:
score_best_individual, parameters_best_individual, res = run_auc_ga(ITERATIONS = 1, POPULATION = 1)

C:\Users\Felps\AppData\Local\Temp\ipykernel_12056\1788604950.py:5: DeprecationWarning: Call to deprecated function (or staticmethod) get_termination. (Please use `from pymoo.termination import get_termination`)
  term = get_termination("n_gen", ITERATIONS)


  0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
#trlist = np.array([])
#for i in range(len(res.history)):
#  trlist = np.append(trlist, -res.history[i].pop.get('F').reshape(-1)) 
#  
#scorelist = []
#
#for i in range(len(trlist)):
##for i in range(1):
#  if i == 0:
#    scorelist.append(trlist[i])
#  elif trlist[i] > scorelist[i-1]:
#    scorelist.append(trlist[i])
#  else:
#    scorelist.append(scorelist[i-1])
#    
#AUC_GA = scorelist
#
#plt.plot(scorelist);

In [ ]:
#tracking = [-np.min(individual.pop.get('F')) for individual in res.history ]
#tracking_GA_AUC = tracking
#plt.plot(tracking)

## Grid Search

### Acurácia

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
#n_possibilities = 2
#
##num_leaves_grid = [i for i in range(num_leaves_min,num_leaves_max, int((num_leaves_max)/13))]
#num_leaves_grid = [i for i in map(lambda x: int(x), np.linspace(num_leaves_min, num_leaves_max, n_possibilities))]
#num_leaves_grid = num_leaves_grid + [100, 50, 75, 125, 11,150]
##print('Num_Leaves_Grid: ')
##print(num_leaves_grid, len(num_leaves_grid))
##print('\n')
#
#
#min_child_samples_grid = [i for i in map(lambda x: int(x), np.linspace(min_child_samples_min, min_child_samples_max, n_possibilities))]
##print('min_child_samples_grid:')
##print(min_child_samples_grid, len(min_child_samples_grid))
##print('\n')
#
#n_estimators_grid = [i for i in map(lambda x: int(x), np.linspace(n_estimators_min, n_estimators_max, n_possibilities))]
##print('n_estimators_grid:')
##print(n_estimators_grid, len(n_estimators_grid))
##print('\n')
#
#learning_rate_grid = np.linspace(learning_rate_min, learning_rate_max, n_possibilities)
##print('learning_rate_grid:')
##print(learning_rate_grid, len(learning_rate_grid))
##print('\n')
#
#subsample_for_bin_grid = [i for i in map(lambda x: int(x), np.linspace(subsample_for_bin_min, subsample_for_bin_max, n_possibilities))]
##print('subsample_for_bin_grid:')
##print(subsample_for_bin_grid, len(subsample_for_bin_grid))
##print('\n')
#
#min_split_gain_grid = np.linspace(min_split_gain_min, min_split_gain_max, n_possibilities)
##print('min_split_gain_grid:')
##print(min_split_gain_grid, len(min_split_gain_grid))
##print('\n')
#
#min_child_weight_grid = np.linspace(min_child_weight_min, min_child_weight_max, n_possibilities)
##print('min_child_weight_grid:')
##print(min_child_weight_grid, len(min_child_weight_grid))
##print('\n')
#
#reg_alpha_grid = np.linspace(reg_alpha_min, reg_alpha_max, n_possibilities)
##print('reg_alpha_grid:')
##print(reg_alpha_grid, len(reg_alpha_grid))
##print('\n')


In [ ]:
#parametros = {'num_leaves': num_leaves_grid, #int
#              'min_child_samples': min_child_samples_grid,#int
#              'n_estimators': n_estimators_grid, #int
#              'learning_rate': learning_rate_grid,
#              'subsample_for_bin': subsample_for_bin_grid, # int
#              'min_split_gain': min_split_gain_grid,
#              'min_child_weight': min_child_weight_grid,
#              'reg_alpha': reg_alpha_grid,
#              'max_depth': [-1],
#              'n_jobs': [-1]}

In [ ]:
#kfold = KFold(n_splits = 3, shuffle = True)
#grid_search = GridSearchCV(estimator = lgb.LGBMClassifier(), param_grid = parametros, cv = kfold, n_jobs= -1, verbose = 3)
#grid_search.fit(x_fruit, y_fruit)
#melhores_parametros = grid_search.best_params_
#melhor_resultado = grid_search.best_score_
##print(melhores_parametros)
##print(melhor_resultado)

In [ ]:
#grid_search.cv_results_.keys()

In [ ]:
#trlist = grid_search.cv_results_['mean_test_score']

In [ ]:
#scorelist = []
#
#for i in range(len(trlist)):
##for i in range(1):
#  if i == 0:
#    scorelist.append(trlist[i])
#  elif trlist[i] > scorelist[i-1]:
#    scorelist.append(trlist[i])
#  else:
#    scorelist.append(scorelist[i-1])

In [ ]:
#tracking_GS_Accuracy = scorelist
#plt.plot(scorelist); 

### F1 Score

In [ ]:
#kfold = KFold(n_splits = 3, shuffle = True)
#grid_search = GridSearchCV(estimator = lgb.LGBMClassifier(), param_grid = parametros, cv = kfold, n_jobs= -1, scoring='f1', verbose = 3)
#grid_search.fit(x_fruit, y_fruit)
#melhores_parametros = grid_search.best_params_
#melhor_resultado = grid_search.best_score_
##print(melhores_parametros)
##print(melhor_resultado)
#
#trlist = grid_search.cv_results_['mean_test_score']
#
#
#scorelist = []
#
#for i in range(len(trlist)):
##for i in range(1):
#  if i == 0:
#    scorelist.append(trlist[i])
#  elif trlist[i] > scorelist[i-1]:
#    scorelist.append(trlist[i])
#  else:
#    scorelist.append(scorelist[i-1])
#    
#tracking_GS_F1 = scorelist
#plt.plot(scorelist); 

### AUC

In [ ]:
#kfold = KFold(n_splits = 3, shuffle = True)
#grid_search = GridSearchCV(estimator = lgb.LGBMClassifier(), param_grid = parametros, cv = kfold, n_jobs= -1, scoring='roc_auc', verbose = 3)
#grid_search.fit(x_fruit, y_fruit)
#melhores_parametros = grid_search.best_params_
#melhor_resultado = grid_search.best_score_
##print(melhores_parametros)
##print(melhor_resultado)
#
#trlist = grid_search.cv_results_['mean_test_score']
#
#
#scorelist = []
#
#for i in range(len(trlist)):
##for i in range(1):
#  if i == 0:
#    scorelist.append(trlist[i])
#  elif trlist[i] > scorelist[i-1]:
#    scorelist.append(trlist[i])
#  else:
#    scorelist.append(scorelist[i-1])
#    
#tracking_GS_AUC = scorelist
#plt.plot(scorelist); 

## Optuna

### Acurácia

In [ ]:
#!pip install optuna

In [ ]:
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)
import sklearn
from sklearn import datasets
def objective_accuracy(trial):
      
      iterations = trial.suggest_int('iterations', iterations_min, iterations_max)  
      learning_rate = trial.suggest_float('learning_rate', learning_rate_min, learning_rate_max)  
      l2_leaf_reg = trial.suggest_float('l2_leaf_reg', l2_leaf_reg_min, l2_leaf_reg_max)  
      bagging_temperature = trial.suggest_float('bagging_temperature', bagging_temperature_min, bagging_temperature_max)  
      random_strength = trial.suggest_float('random_strength', random_strength_min, random_strength_max)  
      best_model_min_trees = trial.suggest_int('best_model_min_trees', best_model_min_trees_min, best_model_min_trees_max)  
      depth = trial.suggest_int('depth', depth_min, depth_max)  
      min_data_in_leaf = trial.suggest_int('min_data_in_leaf', min_data_in_leaf_min, min_data_in_leaf_max)  
      max_leaves = trial.suggest_int('max_leaves', max_leaves_min, max_leaves_max)  
      one_hot_max_size = trial.suggest_int('one_hot_max_size', one_hot_max_size_min, one_hot_max_size_max)  
      rsm = trial.suggest_float('rsm', rsm_min, rsm_max)  
      fold_permutation_block = trial.suggest_int('fold_permutation_block', fold_permutation_block_min, fold_permutation_block_max)  
      leaf_estimation_iterations = trial.suggest_int('leaf_estimation_iterations', leaf_estimation_iterations_min, leaf_estimation_iterations_max)  
      fold_len_multiplier = trial.suggest_float('fold_len_multiplier', fold_len_multiplier_min, fold_len_multiplier_max)  
      boost_from_average = trial.suggest_int('boost_from_average', 0, 1)  
      posterior_sampling = trial.suggest_int('posterior_sampling', 0, 1)  
      allow_const_label = trial.suggest_int('allow_const_label', 0, 1)  
      penalties_coefficient = trial.suggest_float('penalties_coefficient', penalties_coefficient_min, penalties_coefficient_max)  
      model_shrink_rate = trial.suggest_float('model_shrink_rate', model_shrink_rate_min, model_shrink_rate_max)  
      
      
      model_catboost = CatBoostClassifier(
                                    iterations = iterations,
                                    learning_rate = learning_rate,
                                    l2_leaf_reg = l2_leaf_reg,
                                    bagging_temperature = bagging_temperature,
                                    random_strength = random_strength,
                                    best_model_min_trees = best_model_min_trees,
                                    depth = depth,
                                    min_data_in_leaf = min_data_in_leaf,
                                    max_leaves = max_leaves,
                                    one_hot_max_size = one_hot_max_size,
                                    rsm = rsm,
                                    fold_permutation_block = fold_permutation_block,
                                    leaf_estimation_iterations = leaf_estimation_iterations,
                                    fold_len_multiplier = fold_len_multiplier,
                                    #boost_from_average = bool(boost_from_average),
                                    posterior_sampling = bool(posterior_sampling),
                                    allow_const_label = bool(allow_const_label),
                                    penalties_coefficient = penalties_coefficient,
                                    model_shrink_rate = model_shrink_rate,
                                    approx_on_full_history = False,
                                    loss_function='MultiClass',
                                    grow_policy = 'Lossguide',
                                    thread_count = -1
                                    )

      
      kfold = KFold(n_splits = 3, shuffle = True)
      
      return sklearn.model_selection.cross_val_score(model_catboost, x_fruit, y_fruit, n_jobs=-1, cv=kfold).mean()

In [ ]:
def run_optuna_accuracy(n_trials=1024):
    study = optuna.create_study(direction='maximize')
    study.optimize(objective_accuracy, n_trials=n_trials, n_jobs = -1)
    trial = study.best_trial
    #print('Accuracy: {}'.format(trial.value))
    #print("Best hyperparameters: {}".format(trial.params))
    return trial.value, study


In [ ]:
trial, study = run_optuna_accuracy(1)

In [ ]:
trial

0.22052768487551097

In [ ]:
#tracking = []
#for i in range(len(study.trials)):
#  if i == 0: 
#    tracking.append(study.trials[i].value)
#  elif tracking[i-1] > study.trials[i].value:
#    tracking.append(tracking[i-1])
#  else:
#    tracking.append(study.trials[i].value)
##tracking_sorted = sorted(tracking)

In [ ]:
#tracking_Optuna_Accuracy = tracking

In [ ]:
#plt.plot(tracking);
#plt.plot(tracking_sorted) 

In [ ]:
#optuna.visualization.plot_optimization_history(study)

In [ ]:
#optuna.visualization.plot_slice(study)

### F1 Score

In [ ]:
def objective_f1(trial):
  
      iterations = trial.suggest_int('iterations', iterations_min, iterations_max)  
      learning_rate = trial.suggest_float('learning_rate', learning_rate_min, learning_rate_max)  
      l2_leaf_reg = trial.suggest_float('l2_leaf_reg', l2_leaf_reg_min, l2_leaf_reg_max)  
      bagging_temperature = trial.suggest_float('bagging_temperature', bagging_temperature_min, bagging_temperature_max)  
      random_strength = trial.suggest_float('random_strength', random_strength_min, random_strength_max)  
      best_model_min_trees = trial.suggest_int('best_model_min_trees', best_model_min_trees_min, best_model_min_trees_max)  
      depth = trial.suggest_int('depth', depth_min, depth_max)  
      min_data_in_leaf = trial.suggest_int('min_data_in_leaf', min_data_in_leaf_min, min_data_in_leaf_max)  
      max_leaves = trial.suggest_int('max_leaves', max_leaves_min, max_leaves_max)  
      one_hot_max_size = trial.suggest_int('one_hot_max_size', one_hot_max_size_min, one_hot_max_size_max)  
      rsm = trial.suggest_float('rsm', rsm_min, rsm_max)  
      fold_permutation_block = trial.suggest_int('fold_permutation_block', fold_permutation_block_min, fold_permutation_block_max)  
      leaf_estimation_iterations = trial.suggest_int('leaf_estimation_iterations', leaf_estimation_iterations_min, leaf_estimation_iterations_max)  
      fold_len_multiplier = trial.suggest_float('fold_len_multiplier', fold_len_multiplier_min, fold_len_multiplier_max)  
      boost_from_average = trial.suggest_int('boost_from_average', 0, 1)  
      posterior_sampling = trial.suggest_int('posterior_sampling', 0, 1)  
      allow_const_label = trial.suggest_int('allow_const_label', 0, 1)  
      penalties_coefficient = trial.suggest_float('penalties_coefficient', penalties_coefficient_min, penalties_coefficient_max)  
      model_shrink_rate = trial.suggest_float('model_shrink_rate', model_shrink_rate_min, model_shrink_rate_max)  
                                                                
      
      model_catboost = CatBoostClassifier(
                                    iterations = iterations,
                                    learning_rate = learning_rate,
                                    l2_leaf_reg = l2_leaf_reg,
                                    
                                    bagging_temperature = bagging_temperature,
                                    random_strength = random_strength,
                                    best_model_min_trees = best_model_min_trees,
                                    depth = depth,
                                    
                                    min_data_in_leaf = min_data_in_leaf,
                                    max_leaves = max_leaves,
                                    one_hot_max_size = one_hot_max_size,
                                    
                                    rsm = rsm,
                                    fold_permutation_block = fold_permutation_block,
                                    leaf_estimation_iterations = leaf_estimation_iterations,
                                    
                                    
                                    fold_len_multiplier = fold_len_multiplier, #Flag Error
                                    #boost_from_average = bool(boost_from_average), #Flag Error
                                    posterior_sampling = bool(posterior_sampling),#Flag Error
                                    
                                    allow_const_label = bool(allow_const_label),
                                    penalties_coefficient = penalties_coefficient,
                                    model_shrink_rate = model_shrink_rate,
                                    approx_on_full_history = False,
                                    
                                    grow_policy = 'Lossguide',
                                    loss_function='MultiClass',
                                    thread_count = -1
                                    )

      
      kfold = KFold(n_splits = 3, shuffle = True)

      return sklearn.model_selection.cross_val_score(model_catboost, x_fruit, y_fruit, n_jobs=-1, scoring='f1_weighted', cv=kfold).mean()

In [ ]:
def run_optuna_f1(n_trials=1024):
    study = optuna.create_study(direction='maximize')
    study.optimize(objective_f1, n_trials=n_trials, n_jobs = -1)
    trial = study.best_trial
    #print('F1: {}'.format(trial.value))
    #print("Best hyperparameters: {}".format(trial.params))
    return trial.value, study

In [ ]:
trial, study = run_optuna_f1(1)

In [ ]:
#tracking = []
#for i in range(len(study.trials)):
#  if i == 0: 
#    tracking.append(study.trials[i].value)
#  elif tracking[i-1] > study.trials[i].value:
#    tracking.append(tracking[i-1])
#  else:
#    tracking.append(study.trials[i].value)
##tracking_sorted = sorted(tracking)
#
#tracking_Optuna_F1 = tracking
#
#plt.plot(tracking);
##plt.plot(tracking_sorted) 

### AUC

In [ ]:
def objective_auc(trial):
  
      iterations = trial.suggest_int('iterations', iterations_min, iterations_max)  
      learning_rate = trial.suggest_float('learning_rate', learning_rate_min, learning_rate_max)  
      l2_leaf_reg = trial.suggest_float('l2_leaf_reg', l2_leaf_reg_min, l2_leaf_reg_max)  
      bagging_temperature = trial.suggest_float('bagging_temperature', bagging_temperature_min, bagging_temperature_max)  
      random_strength = trial.suggest_float('random_strength', random_strength_min, random_strength_max)  
      best_model_min_trees = trial.suggest_int('best_model_min_trees', best_model_min_trees_min, best_model_min_trees_max)  
      depth = trial.suggest_int('depth', depth_min, depth_max)  
      min_data_in_leaf = trial.suggest_int('min_data_in_leaf', min_data_in_leaf_min, min_data_in_leaf_max)  
      max_leaves = trial.suggest_int('max_leaves', max_leaves_min, max_leaves_max)  
      one_hot_max_size = trial.suggest_int('one_hot_max_size', one_hot_max_size_min, one_hot_max_size_max)  
      rsm = trial.suggest_float('rsm', rsm_min, rsm_max)  
      fold_permutation_block = trial.suggest_int('fold_permutation_block', fold_permutation_block_min, fold_permutation_block_max)  
      leaf_estimation_iterations = trial.suggest_int('leaf_estimation_iterations', leaf_estimation_iterations_min, leaf_estimation_iterations_max)  
      fold_len_multiplier = trial.suggest_float('fold_len_multiplier', fold_len_multiplier_min, fold_len_multiplier_max)  
      boost_from_average = trial.suggest_int('boost_from_average', 0, 1)  
      posterior_sampling = trial.suggest_int('posterior_sampling', 0, 1)  
      allow_const_label = trial.suggest_int('allow_const_label', 0, 1)  
      penalties_coefficient = trial.suggest_float('penalties_coefficient', penalties_coefficient_min, penalties_coefficient_max)  
      model_shrink_rate = trial.suggest_float('model_shrink_rate', model_shrink_rate_min, model_shrink_rate_max)  
                                                                
      
      model_catboost = CatBoostClassifier(
                                    iterations = iterations,
                                    learning_rate = learning_rate,
                                    l2_leaf_reg = l2_leaf_reg,
                                    
                                    bagging_temperature = bagging_temperature,
                                    random_strength = random_strength,
                                    best_model_min_trees = best_model_min_trees,
                                    depth = depth,
                                    
                                    min_data_in_leaf = min_data_in_leaf,
                                    max_leaves = max_leaves,
                                    one_hot_max_size = one_hot_max_size,
                                    
                                    rsm = rsm,
                                    fold_permutation_block = fold_permutation_block,
                                    leaf_estimation_iterations = leaf_estimation_iterations,
                                    
                                    
                                    fold_len_multiplier = fold_len_multiplier, #Flag Error
                                    #boost_from_average = bool(boost_from_average), #Flag Error
                                    posterior_sampling = bool(posterior_sampling),#Flag 
                                    
                                    allow_const_label = bool(allow_const_label),
                                    penalties_coefficient = penalties_coefficient,
                                    model_shrink_rate = model_shrink_rate,
                                    approx_on_full_history = False,
                                    
                                    grow_policy = 'Lossguide',
                                    loss_function='MultiClass',
                                    thread_count = -1
                                    )

      
      kfold = KFold(n_splits = 3, shuffle = True)


      return sklearn.model_selection.cross_val_score(model_catboost, x_fruit, y_fruit, n_jobs=-1, scoring='roc_auc_ovr_weighted', cv=kfold).mean()

In [ ]:
def run_optuna_auc(n_trials=1024):
    study = optuna.create_study(direction='maximize')
    study.optimize(objective_auc, n_trials=n_trials, n_jobs = -1)
    trial = study.best_trial
    #print('AUC: {}'.format(trial.value))
    #print("Best hyperparameters: {}".format(trial.params))
    return trial.value, study

In [ ]:
trial, study = run_optuna_auc(1)

In [ ]:
#tracking = []
#for i in range(len(study.trials)):
#  if i == 0: 
#    tracking.append(study.trials[i].value)
#  elif tracking[i-1] > study.trials[i].value:
#    tracking.append(tracking[i-1])
#  else:
#    tracking.append(study.trials[i].value)
##tracking_sorted = sorted(tracking)
#
#tracking_Optuna_AUC = tracking
#
#plt.plot(tracking);
##plt.plot(tracking_sorted) 

# Análise Comparativa

In [ ]:
#fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,5))
#
#ax1.set_title('PSO X Gerações')
#ax2.set_title('PSO X Avaliações')
#
#ax1.plot(tracking_PSO_Accuracy)
#ax2.plot(Accuracy_PSO)
#plt.show()

In [ ]:
#fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,5))
#
#ax1.set_title('GA X Gerações')
#ax2.set_title('GA X Avaliações')
#
#ax1.plot(tracking_GA_Accuracy)
#ax2.plot(Accuracy_GA)
#plt.show()

Gráficos Comparativos Acurácia

In [ ]:
#from IPython.core.pylabtools import figsize
#from matplotlib.pyplot import figure
#import matplotlib.patches as mpatches
#
#plt.figure(figsize=(12, 6))
#red_patch = mpatches.Patch(color='red', label='PSO')
#blue_patch = mpatches.Patch(color='blue', label='GA')
#green_patch = mpatches.Patch(color='green', label='Optuna')
##black_patch = mpatches.Patch(color='black', label='GridSearch')
#
#plt.legend(handles=[red_patch, blue_patch, green_patch])
#plt.plot(Accuracy_GA, color = 'blue', linewidth=1, linestyle='-', )
#plt.plot(Accuracy_PSO, color = 'red', linewidth=1, linestyle='-')
#plt.plot(tracking_Optuna_Accuracy, color = 'green', linewidth=1, linestyle='-')
##plt.plot(tracking_GS, color = 'black', linewidth=3, linestyle='-')
#plt.show()

In [ ]:
#fig, ax = plt.subplots(2, 2, figsize=(25,15))
#
#ax[0, 0].set_title(f'Genetic Algorithm (GA) - Accuracy: {max(tracking_GA_Accuracy)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#ax[0, 1].set_title(f'Particle Swarm Optimization (PSO) - Accuracy: {max(tracking_PSO_Accuracy)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#
#ax[1, 0].set_title(f'Grid Search - Accuracy: {max(tracking_GS_Accuracy)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#ax[1, 1].set_title(f'Optuna - Accuracy: {max(tracking_Optuna_Accuracy)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#
#ax[0,0].plot(tracking_GA_Accuracy, color = 'blue', linewidth=3, linestyle='-', )
#ax[0,1].plot(tracking_PSO_Accuracy, color = 'red', linewidth=3, linestyle='-')
#
#ax[1,0].plot(tracking_GS_Accuracy, color = 'black', linewidth=3, linestyle='-')
#ax[1,1].plot(tracking_Optuna_Accuracy, color = 'green', linewidth=3, linestyle='-')


F1 score

In [ ]:
#plt.figure(figsize=(12, 6))
#red_patch = mpatches.Patch(color='red', label='PSO')
#blue_patch = mpatches.Patch(color='blue', label='GA')
#green_patch = mpatches.Patch(color='green', label='Optuna')
##black_patch = mpatches.Patch(color='black', label='GridSearch')
#
#plt.legend(handles=[red_patch, blue_patch, green_patch])
#plt.plot(F1_GA, color = 'blue', linewidth=1, linestyle='-', )
#plt.plot(F1_PSO, color = 'red', linewidth=1, linestyle='-')
#plt.plot(tracking_Optuna_F1, color = 'green', linewidth=1, linestyle='-')
##plt.plot(tracking_GS, color = 'black', linewidth=3, linestyle='-')
#plt.show()

In [ ]:
#fig, ax = plt.subplots(2, 2, figsize=(25,15))
#
#ax[0, 0].set_title(f'Genetic Algorithm (GA) - Accuracy: {max(tracking_GA_F1)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#ax[0, 1].set_title(f'Particle Swarm Optimization (PSO) - Accuracy: {max(tracking_PSO_F1)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#
#ax[1, 0].set_title(f'Grid Search - Accuracy: {max(tracking_GS_F1)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#ax[1, 1].set_title(f'Optuna - Accuracy: {max(tracking_Optuna_F1)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#
#ax[0,0].plot(tracking_GA_F1, color = 'blue', linewidth=3, linestyle='-', )
#ax[0,1].plot(tracking_PSO_F1, color = 'red', linewidth=3, linestyle='-')
#
#ax[1,0].plot(tracking_GS_F1, color = 'black', linewidth=3, linestyle='-')
#ax[1,1].plot(tracking_Optuna_F1, color = 'green', linewidth=3, linestyle='-')


ROC AUC

In [ ]:
#plt.figure(figsize=(12, 6))
#red_patch = mpatches.Patch(color='red', label='PSO')
#blue_patch = mpatches.Patch(color='blue', label='GA')
#green_patch = mpatches.Patch(color='green', label='Optuna')
##black_patch = mpatches.Patch(color='black', label='GridSearch')
#
#plt.legend(handles=[red_patch, blue_patch, green_patch])
#plt.plot(AUC_GA, color = 'blue', linewidth=1, linestyle='-', )
#plt.plot(AUC_PSO, color = 'red', linewidth=1, linestyle='-')
#plt.plot(tracking_Optuna_AUC, color = 'green', linewidth=1, linestyle='-')
##plt.plot(tracking_GS, color = 'black', linewidth=3, linestyle='-')
#plt.show()

In [ ]:
#fig, ax = plt.subplots(2, 2, figsize=(25,15))
#
#ax[0, 0].set_title(f'Genetic Algorithm (GA) - Accuracy: {max(tracking_GA_AUC)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#ax[0, 1].set_title(f'Particle Swarm Optimization (PSO) - Accuracy: {max(tracking_PSO_AUC)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#
#ax[1, 0].set_title(f'Grid Search - Accuracy: {max(tracking_GS_AUC)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#ax[1, 1].set_title(f'Optuna - Accuracy: {max(tracking_Optuna_AUC)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#
#ax[0,0].plot(tracking_GA_AUC, color = 'blue', linewidth=3, linestyle='-', )
#ax[0,1].plot(tracking_PSO_AUC, color = 'red', linewidth=3, linestyle='-')
#
#ax[1,0].plot(tracking_GS_AUC, color = 'black', linewidth=3, linestyle='-')
#ax[1,1].plot(tracking_Optuna_AUC, color = 'green', linewidth=3, linestyle='-')
#

In [ ]:
from scipy.stats import uniform
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

def random_search_int_range(min_l, max_l):
    return np.arange(randint.ppf(0.01, min_l, max_l + 2),randint.ppf(0.99, min_l, max_l + 2))


def run_random_search_accuracy(iterations = 1024):
    param_distribution = {
    'iterations' : random_search_int_range(iterations_min, iterations_max),
    'learning_rate' : uniform(learning_rate_min, learning_rate_max),
    'l2_leaf_reg' : uniform(l2_leaf_reg_min, l2_leaf_reg_max),
    'bagging_temperature' : uniform(bagging_temperature_min, bagging_temperature_max),
    'random_strength' : uniform(random_strength_min, random_strength_max),
    'best_model_min_trees' : random_search_int_range(best_model_min_trees_min, best_model_min_trees_max),
    'depth' : random_search_int_range(depth_min, depth_max),
    'min_data_in_leaf' : random_search_int_range(min_data_in_leaf_min, min_data_in_leaf_max),
    'max_leaves' : random_search_int_range(max_leaves_min, max_leaves_max),
    'one_hot_max_size' : random_search_int_range(one_hot_max_size_min, one_hot_max_size_max),
    'rsm' : uniform(rsm_min + 0.00001, rsm_max),
    'fold_permutation_block' : random_search_int_range(fold_permutation_block_min, fold_permutation_block_max),
    'leaf_estimation_iterations' : random_search_int_range(leaf_estimation_iterations_min, leaf_estimation_iterations_max),
    'fold_len_multiplier' : uniform(fold_len_multiplier_min, fold_len_multiplier_max),
    'posterior_sampling' : [posterior_sampling_min, posterior_sampling_max],
    'allow_const_label' : [allow_const_label_min, allow_const_label_max],
    'penalties_coefficient' : uniform(penalties_coefficient_min, penalties_coefficient_max),
    'model_shrink_rate' : uniform(model_shrink_rate_min, model_shrink_rate_max),
    'approx_on_full_history' : [False],
    'grow_policy' : ['Lossguide'],
    'loss_function' : ['MultiClass'],
    'thread_count' :  [-1]
    }




    kfold = KFold(n_splits = 3, shuffle = True)

    random_search_accuracy = RandomizedSearchCV(CatBoostClassifier(), param_distribution, n_iter = iterations, n_jobs = 10, cv=kfold, verbose=False)
    random_search_accuracy.fit(x_fruit, y_fruit)

    return random_search_accuracy.best_score_

In [ ]:
from scipy.stats import uniform
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

def random_search_int_range(min_l, max_l):
    return np.arange(randint.ppf(0.01, min_l, max_l + 2),randint.ppf(0.99, min_l, max_l + 2))


def run_random_search_f1(iterations = 1024):
    param_distribution = {
    'iterations' : random_search_int_range(iterations_min, iterations_max),
    'learning_rate' : uniform(learning_rate_min, learning_rate_max),
    'l2_leaf_reg' : uniform(l2_leaf_reg_min, l2_leaf_reg_max),
    'bagging_temperature' : uniform(bagging_temperature_min, bagging_temperature_max),
    'random_strength' : uniform(random_strength_min, random_strength_max),
    'best_model_min_trees' : random_search_int_range(best_model_min_trees_min, best_model_min_trees_max),
    'depth' : random_search_int_range(depth_min, depth_max),
    'min_data_in_leaf' : random_search_int_range(min_data_in_leaf_min, min_data_in_leaf_max),
    'max_leaves' : random_search_int_range(max_leaves_min, max_leaves_max),
    'one_hot_max_size' : random_search_int_range(one_hot_max_size_min, one_hot_max_size_max),
    'rsm' : uniform(rsm_min + 0.00001, rsm_max),
    'fold_permutation_block' : random_search_int_range(fold_permutation_block_min, fold_permutation_block_max),
    'leaf_estimation_iterations' : random_search_int_range(leaf_estimation_iterations_min, leaf_estimation_iterations_max),
    'fold_len_multiplier' : uniform(fold_len_multiplier_min, fold_len_multiplier_max),
    'posterior_sampling' : [posterior_sampling_min, posterior_sampling_max],
    'allow_const_label' : [allow_const_label_min, allow_const_label_max],
    'penalties_coefficient' : uniform(penalties_coefficient_min, penalties_coefficient_max),
    'model_shrink_rate' : uniform(model_shrink_rate_min, model_shrink_rate_max),
    'approx_on_full_history' : [False],
    'grow_policy' : ['Lossguide'],
    'loss_function' : ['MultiClass'],
    'thread_count' :  [-1]
    }




    kfold = KFold(n_splits = 3, shuffle = True)

    random_search_accuracy = RandomizedSearchCV(CatBoostClassifier(), param_distribution, n_iter = iterations, n_jobs = 10, cv=kfold, verbose=False, scoring='f1_weighted')
    random_search_accuracy.fit(x_fruit, y_fruit)

    return random_search_accuracy.best_score_

In [ ]:
from scipy.stats import uniform
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

def random_search_int_range(min_l, max_l):
    return np.arange(randint.ppf(0.01, min_l, max_l + 2),randint.ppf(0.99, min_l, max_l + 2))


def run_random_search_auc(iterations = 1024):
    param_distribution = {
    'iterations' : random_search_int_range(iterations_min, iterations_max),
    'learning_rate' : uniform(learning_rate_min, learning_rate_max),
    'l2_leaf_reg' : uniform(l2_leaf_reg_min, l2_leaf_reg_max),
    'bagging_temperature' : uniform(bagging_temperature_min, bagging_temperature_max),
    'random_strength' : uniform(random_strength_min, random_strength_max),
    'best_model_min_trees' : random_search_int_range(best_model_min_trees_min, best_model_min_trees_max),
    'depth' : random_search_int_range(depth_min, depth_max),
    'min_data_in_leaf' : random_search_int_range(min_data_in_leaf_min, min_data_in_leaf_max),
    'max_leaves' : random_search_int_range(max_leaves_min, max_leaves_max),
    'one_hot_max_size' : random_search_int_range(one_hot_max_size_min, one_hot_max_size_max),
    'rsm' : uniform(rsm_min + 0.00001, rsm_max),
    'fold_permutation_block' : random_search_int_range(fold_permutation_block_min, fold_permutation_block_max),
    'leaf_estimation_iterations' : random_search_int_range(leaf_estimation_iterations_min, leaf_estimation_iterations_max),
    'fold_len_multiplier' : uniform(fold_len_multiplier_min, fold_len_multiplier_max),
    'posterior_sampling' : [posterior_sampling_min, posterior_sampling_max],
    'allow_const_label' : [allow_const_label_min, allow_const_label_max],
    'penalties_coefficient' : uniform(penalties_coefficient_min, penalties_coefficient_max),
    'model_shrink_rate' : uniform(model_shrink_rate_min, model_shrink_rate_max),
    'approx_on_full_history' : [False],
    'grow_policy' : ['Lossguide'],
    'loss_function' : ['MultiClass'],
    'thread_count' :  [-1]
    }




    kfold = KFold(n_splits = 3, shuffle = True)

    random_search_accuracy = RandomizedSearchCV(CatBoostClassifier(), param_distribution, n_iter = iterations, n_jobs = 10, cv=kfold, verbose=False, scoring='roc_auc_ovr_weighted')
    random_search_accuracy.fit(x_fruit, y_fruit)

    return random_search_accuracy.best_score_

# Gerando Dados Para Análise 

In [ ]:
from tqdm import tqdm
import pandas as pd
from os import system

def full_run(iterations):
    filename = './Catboost_fruit_data.csv'

    try:
        Catboost_data = pd.read_csv(filename)
    except:
        open(filename, "a")
        Catboost_data = pd.DataFrame(columns=['Algorithm', 'Accuracy', 'F1', 'AUC'])


    for i in tqdm(range(iterations)):
        accuracy_pso, _, _ = run_accuracy_pso()
        f1_pso, _, _ = run_f1_pso()
        auc_pso, _, _ = run_auc_pso()
        temp = pd.DataFrame({'Algorithm' : ['PSO'], 
                            'Accuracy' :[-accuracy_pso[0]], 
                            'F1' : [-f1_pso[0]], 
                            'AUC' : [-auc_pso[0]]})
        #Catboost_data = Catboost_data.append(temp, ignore_index = True)
        Catboost_data = pd.concat([Catboost_data, temp], ignore_index=True)
        Catboost_data[['Algorithm', 'Accuracy', 'F1', 'AUC']].to_csv(filename)
        
        accuracy_ga, _, _ = run_accuracy_ga()
        f1_ga, _, _ = run_f1_ga()
        auc_ga, _, _ = run_auc_ga()
        temp = pd.DataFrame({'Algorithm' : ['GA'], 
                            'Accuracy' :[-accuracy_ga[0]], 
                            'F1' : [-f1_ga[0]], 
                            'AUC' : [-auc_ga[0]]})
        #Catboost_data = Catboost_data.append(temp, ignore_index = True)
        Catboost_data = pd.concat([Catboost_data, temp], ignore_index=True)
        Catboost_data[['Algorithm', 'Accuracy', 'F1', 'AUC']].to_csv(filename)
        
        accuracy_optuna, _ = run_optuna_accuracy()
        f1_optuna, _ = run_optuna_f1()
        auc_optuna, _ = run_optuna_auc()
        temp = pd.DataFrame({'Algorithm' : ['Optuna'], 
                            'Accuracy' :[accuracy_optuna], 
                            'F1' : [f1_optuna], 
                            'AUC' : [auc_optuna]})
        #Catboost_data = Catboost_data.append(temp, ignore_index = True)
        Catboost_data = pd.concat([Catboost_data, temp], ignore_index=True)
        Catboost_data[['Algorithm', 'Accuracy', 'F1', 'AUC']].to_csv(filename)

In [ ]:
from tqdm import tqdm
import pandas as pd
from os import system

def full_run_optuna(iterations):
    filename = './Catboost_wine_data.csv'

    try:
        Catboost_data = pd.read_csv(filename)
    except:
        open(filename, "a")
        Catboost_data = pd.DataFrame(columns=['Algorithm', 'Accuracy', 'F1', 'AUC'])


    for i in tqdm(range(iterations)):      
        accuracy_optuna, _ = run_optuna_accuracy()
        f1_optuna, _ = run_optuna_f1()
        auc_optuna, _ = run_optuna_auc()
        temp = pd.DataFrame({'Algorithm' : ['Optuna'], 
                            'Accuracy' :[accuracy_optuna], 
                            'F1' : [f1_optuna], 
                            'AUC' : [auc_optuna]})
        #Catboost_data = Catboost_data.append(temp, ignore_index = True)
        Catboost_data = pd.concat([Catboost_data, temp], ignore_index=True)
        Catboost_data[['Algorithm', 'Accuracy', 'F1', 'AUC']].to_csv(filename)

In [ ]:
from tqdm import tqdm
import pandas as pd
from os import system

def full_run_ga(iterations):
    filename = './Catboost_fruit_data.csv'

    try:
        Catboost_data = pd.read_csv(filename)
    except:
        open(filename, "a")
        Catboost_data = pd.DataFrame(columns=['Algorithm', 'Accuracy', 'F1', 'AUC'])


    for i in tqdm(range(iterations)):

        accuracy_ga, _, _ = run_accuracy_ga()
        f1_ga, _, _ = run_f1_ga()
        auc_ga, _, _ = run_auc_ga()
        temp = pd.DataFrame({'Algorithm' : ['GA'], 
                            'Accuracy' :[-accuracy_ga[0]], 
                            'F1' : [-f1_ga[0]], 
                            'AUC' : [-auc_ga[0]]})
        #Catboost_data = Catboost_data.append(temp, ignore_index = True)
        Catboost_data = pd.concat([Catboost_data, temp], ignore_index=True)
        Catboost_data[['Algorithm', 'Accuracy', 'F1', 'AUC']].to_csv(filename)
        

In [ ]:
from tqdm import tqdm
import pandas as pd
from os import system

def full_run_optuna(iterations):
    filename = './Catboost_fruit_data.csv'

    try:
        Catboost_data = pd.read_csv(filename)
    except:
        open(filename, "a")
        Catboost_data = pd.DataFrame(columns=['Algorithm', 'Accuracy', 'F1', 'AUC'])


    for i in tqdm(range(iterations)):      
        accuracy_optuna, _ = run_optuna_accuracy()
        f1_optuna, _ = run_optuna_f1()
        auc_optuna, _ = run_optuna_auc()
        temp = pd.DataFrame({'Algorithm' : ['Optuna'], 
                            'Accuracy' :[accuracy_optuna], 
                            'F1' : [f1_optuna], 
                            'AUC' : [auc_optuna]})
        #Catboost_data = Catboost_data.append(temp, ignore_index = True)
        Catboost_data = pd.concat([Catboost_data, temp], ignore_index=True)
        Catboost_data[['Algorithm', 'Accuracy', 'F1', 'AUC']].to_csv(filename)

In [52]:
from tqdm import tqdm
import pandas as pd
from os import system

def full_run_pso(iterations):
    filename = './Catboost_fruit_data.csv'

    try:
        Catboost_data = pd.read_csv(filename)
    except:
        open(filename, "a")
        Catboost_data = pd.DataFrame(columns=['Algorithm', 'Accuracy', 'F1', 'AUC'])


    for i in tqdm(range(iterations)):
        accuracy_pso = run_accuracy_pso()
        f1_pso = run_f1_pso()
        auc_pso = run_auc_pso()
        temp = pd.DataFrame({'Algorithm' : ['PSO'], 
                            'Accuracy' :[accuracy_pso], 
                            'F1' : [f1_pso], 
                            'AUC' : [auc_pso]})
        #Catboost_data = Catboost_data.append(temp, ignore_index = True)
        Catboost_data = pd.concat([Catboost_data, temp], ignore_index=True)
        Catboost_data[['Algorithm', 'Accuracy', 'F1', 'AUC']].to_csv(filename)
        

In [53]:
full_run_pso(30)

2023-05-30 21:07:49,441 - pyswarms.single.global_best - INFO - Optimize for 32 iters with {'c1': 1.5, 'c2': 1.5, 'w': 0.5}

































































pyswarms.single.global_best: 100%|██████████|32/32, best_cost=-.222
2023-05-30 21:08:28,607 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -0.22168539325842698, best pos: [7.67846774e+01 1.53267850e-01 5.53652563e-01 6.55494417e+03
 1.50301888e+00 6.32392282e+00 4.99891376e+00 7.49827281e+01
 2.02688005e+00 1.87048757e+01 2.57569314e-01 9.31133861e+01
 4.63023857e+01 3.57101811e+00 8.95099010e+00 5.83772977e-01]
2023-05-30 21:08:28,613 - pyswarms.single.global_best - INFO - Optimize for 32 iters with {'c1': 1.5, 'c2': 1.5, 'w': 0.5}

































































pyswarms.single.global_best: 100%|██████████|32/32, best_cost=-.085
2023-05-30 21:08:48,312 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -0.085049646002

In [ ]:
def full_run_random_searh(iterations):
    filename = './Catboost_fruit_data.csv'

    try:
        Catboost_data = pd.read_csv(filename)
    except:
        open(filename, "a")
        Catboost_data = pd.DataFrame(columns=['Algorithm', 'Accuracy', 'F1', 'AUC'])


    for i in tqdm(range(iterations)):

        accuracy_rs = run_random_search_accuracy()
        f1_rs = run_random_search_f1()
        auc_rs = run_random_search_auc()
        temp = pd.DataFrame({'Algorithm' : ['Random Search'], 
                            'Accuracy' :[accuracy_rs], 
                            'F1' : [f1_rs], 
                            'AUC' : [auc_rs]})
        #Catboost_data = Catboost_data.append(temp, ignore_index = True)
        Catboost_data = pd.concat([Catboost_data, temp], ignore_index=True)
        Catboost_data[['Algorithm', 'Accuracy', 'F1', 'AUC']].to_csv(filename)

In [ ]:
full_run_random_searh(1)

  0%|          | 0/1 [00:00<?, ?it/s]

0:	learn: 1.4931873	total: 3.66ms	remaining: 296ms
1:	learn: 1.2037301	total: 7.09ms	remaining: 284ms
2:	learn: 1.1618254	total: 10.9ms	remaining: 286ms
3:	learn: 0.9395561	total: 14.2ms	remaining: 277ms
4:	learn: 0.9280262	total: 17.6ms	remaining: 270ms
5:	learn: 0.8329135	total: 21.1ms	remaining: 267ms
6:	learn: 0.8071646	total: 24.4ms	remaining: 262ms
7:	learn: 0.7141258	total: 28.3ms	remaining: 262ms
8:	learn: 0.5985504	total: 31.5ms	remaining: 255ms
9:	learn: 0.5947802	total: 35.3ms	remaining: 254ms
10:	learn: 0.5912574	total: 38.9ms	remaining: 251ms
11:	learn: 0.5663908	total: 42.5ms	remaining: 248ms
12:	learn: 0.5373099	total: 46.2ms	remaining: 245ms
13:	learn: 0.4990285	total: 49.9ms	remaining: 242ms
14:	learn: 0.4720677	total: 53.9ms	remaining: 241ms
15:	learn: 0.4660290	total: 57.4ms	remaining: 237ms
16:	learn: 0.4611349	total: 60.9ms	remaining: 233ms
17:	learn: 0.4263301	total: 64.3ms	remaining: 229ms
18:	learn: 0.4217923	total: 67.7ms	remaining: 225ms
19:	learn: 0.4111042	t

C:\Users\Felps\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
3 fits failed out of a total of 3072.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Felps\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Felps\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-pack

0:	learn: 1.5978257	total: 16.5ms	remaining: 1.29s
1:	learn: 1.5100664	total: 33.3ms	remaining: 1.28s
2:	learn: 1.3004012	total: 50.8ms	remaining: 1.29s
3:	learn: 1.1192124	total: 69.5ms	remaining: 1.3s
4:	learn: 1.0648865	total: 86.4ms	remaining: 1.28s
5:	learn: 0.9033424	total: 103ms	remaining: 1.25s
6:	learn: 0.9154434	total: 112ms	remaining: 1.15s
7:	learn: 0.9371350	total: 120ms	remaining: 1.07s
8:	learn: 0.8854856	total: 137ms	remaining: 1.06s
9:	learn: 0.8895915	total: 155ms	remaining: 1.07s
10:	learn: 0.9126258	total: 164ms	remaining: 1.01s
11:	learn: 0.9337752	total: 180ms	remaining: 1.01s
12:	learn: 0.9262116	total: 197ms	remaining: 1s
13:	learn: 0.8900828	total: 214ms	remaining: 992ms
14:	learn: 0.8630855	total: 230ms	remaining: 983ms
15:	learn: 0.8635263	total: 247ms	remaining: 974ms
16:	learn: 0.8899986	total: 256ms	remaining: 934ms
17:	learn: 0.8876465	total: 273ms	remaining: 925ms
18:	learn: 0.8876227	total: 281ms	remaining: 888ms
19:	learn: 0.9027680	total: 298ms	remain

100%|██████████| 1/1 [07:42<00:00, 462.20s/it]


In [ ]:
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)

  0%|          | 0/1 [00:00<?, ?it/s]C:\Users\Felps\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
3 fits failed out of a total of 3072.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Felps\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Felps\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.1

0:	learn: 1.7592162	total: 9.85ms	remaining: 295ms
1:	learn: 1.6465123	total: 20.1ms	remaining: 291ms
2:	learn: 1.4116676	total: 29.7ms	remaining: 277ms
3:	learn: 1.3124983	total: 39.4ms	remaining: 266ms
4:	learn: 1.0976585	total: 49.2ms	remaining: 256ms
5:	learn: 1.0043389	total: 58.9ms	remaining: 246ms
6:	learn: 0.9619802	total: 68.7ms	remaining: 236ms
7:	learn: 0.9470580	total: 78.5ms	remaining: 226ms
8:	learn: 0.8710348	total: 88.1ms	remaining: 215ms
9:	learn: 0.8358247	total: 98ms	remaining: 206ms
10:	learn: 0.7922264	total: 108ms	remaining: 196ms
11:	learn: 0.7820227	total: 113ms	remaining: 179ms
12:	learn: 0.7636137	total: 123ms	remaining: 170ms
13:	learn: 0.7445491	total: 133ms	remaining: 161ms
14:	learn: 0.7492183	total: 138ms	remaining: 147ms
15:	learn: 0.7327476	total: 148ms	remaining: 138ms
16:	learn: 0.7150518	total: 157ms	remaining: 130ms
17:	learn: 0.7024512	total: 163ms	remaining: 118ms
18:	learn: 0.6956773	total: 172ms	remaining: 109ms
19:	learn: 0.6878774	total: 182ms

100%|██████████| 1/1 [07:17<00:00, 437.31s/it]


61:	learn: 1.1961878	total: 649ms	remaining: 94.3ms
62:	learn: 1.2049467	total: 660ms	remaining: 83.9ms
63:	learn: 1.2055820	total: 672ms	remaining: 73.5ms
64:	learn: 1.2054979	total: 683ms	remaining: 63ms
65:	learn: 1.2153226	total: 694ms	remaining: 52.6ms
66:	learn: 1.2248784	total: 705ms	remaining: 42.1ms
67:	learn: 1.1998029	total: 717ms	remaining: 31.6ms
68:	learn: 1.2087054	total: 728ms	remaining: 21.1ms
69:	learn: 1.2083418	total: 739ms	remaining: 10.6ms
70:	learn: 1.2154558	total: 750ms	remaining: 0us


  0%|          | 0/1 [00:00<?, ?it/s]C:\Users\Felps\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
3 fits failed out of a total of 3072.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Felps\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Felps\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.1

...

60:	learn: 0.2806886	total: 424ms	remaining: 62.5ms
61:	learn: 0.2764766	total: 430ms	remaining: 55.5ms
62:	learn: 0.2758205	total: 437ms	remaining: 48.6ms
63:	learn: 0.2754038	total: 444ms	remaining: 41.7ms
64:	learn: 0.2733804	total: 451ms	remaining: 34.7ms
65:	learn: 0.2668612	total: 458ms	remaining: 27.8ms
66:	learn: 0.2633575	total: 465ms	remaining: 20.8ms
67:	learn: 0.2618964	total: 472ms	remaining: 13.9ms
68:	learn: 0.2609620	total: 479ms	remaining: 6.94ms
69:	learn: 0.2600073	total: 486ms	remaining: 0us


  0%|          | 0/1 [00:00<?, ?it/s]

0:	learn: 1.5978662	total: 4.29ms	remaining: 292ms
1:	learn: 1.3228113	total: 8.52ms	remaining: 285ms
2:	learn: 1.3138264	total: 12.4ms	remaining: 273ms
3:	learn: 1.1928747	total: 16.4ms	remaining: 267ms
4:	learn: 1.0546864	total: 20.7ms	remaining: 265ms
5:	learn: 1.0522407	total: 24.9ms	remaining: 262ms
6:	learn: 0.9971774	total: 28.9ms	remaining: 256ms
7:	learn: 0.8579568	total: 33ms	remaining: 251ms
8:	learn: 0.8367156	total: 37.2ms	remaining: 248ms
9:	learn: 0.7938687	total: 41.7ms	remaining: 246ms
10:	learn: 0.7512035	total: 46ms	remaining: 242ms
11:	learn: 0.6180903	total: 50.2ms	remaining: 239ms
12:	learn: 0.6169098	total: 54.1ms	remaining: 233ms
13:	learn: 0.6078951	total: 58.3ms	remaining: 229ms
14:	learn: 0.5492936	total: 62.9ms	remaining: 226ms
15:	learn: 0.5490441	total: 67.2ms	remaining: 223ms
16:	learn: 0.5418536	total: 71.7ms	remaining: 219ms
17:	learn: 0.5098297	total: 76ms	remaining: 215ms
18:	learn: 0.5097089	total: 80.1ms	remaining: 211ms
19:	learn: 0.5066810	total: 

100%|██████████| 1/1 [07:14<00:00, 434.96s/it]

38:	learn: 1.8663096	total: 210ms	remaining: 124ms
39:	learn: 1.8663096	total: 216ms	remaining: 119ms
40:	learn: 1.8663096	total: 221ms	remaining: 113ms
41:	learn: 1.8663096	total: 227ms	remaining: 108ms
42:	learn: 1.8663096	total: 232ms	remaining: 103ms
43:	learn: 1.8663096	total: 238ms	remaining: 97.2ms
44:	learn: 1.8663096	total: 243ms	remaining: 91.8ms
45:	learn: 1.8663096	total: 248ms	remaining: 86.3ms
46:	learn: 1.8663096	total: 253ms	remaining: 80.9ms
47:	learn: 1.8663096	total: 259ms	remaining: 75.5ms
48:	learn: 1.8663096	total: 264ms	remaining: 70.1ms
49:	learn: 1.8663096	total: 270ms	remaining: 64.7ms
50:	learn: 1.8663096	total: 275ms	remaining: 59.4ms
51:	learn: 1.8663096	total: 281ms	remaining: 54ms
52:	learn: 1.8663096	total: 286ms	remaining: 48.6ms
53:	learn: 1.8663096	total: 292ms	remaining: 43.2ms
54:	learn: 1.8663096	total: 297ms	remaining: 37.8ms
55:	learn: 1.8663096	total: 303ms	remaining: 32.4ms
56:	learn: 1.8663096	total: 308ms	remaining: 27ms
57:	learn: 1.8663096	